<a href="https://colab.research.google.com/github/adigenova/uohpmd/blob/main/code/Haddop_nextflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hadoop

## Calcular el promedio

In [1]:
%%writefile mapper1.py
#!/usr/bin/env python
import sys

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    number = float(line.strip())
    # Emite el número como clave y valor
    print("Total", number)
    print("Count", 1)

Writing mapper1.py


In [16]:
!chmod 777 /content/mapper1.py /content/reducer1.py

In [15]:
%%writefile reducer1.py
#!/usr/bin/env python
import sys

current_key = None
total = 0
count = 0

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    key, value = line.strip().split(" ")
    value = float(value)

    if key == "Total":
        total += value
    elif key == "Count":
        count += int(value)

# Calcula el promedio
average = total / count
print("Average", average)


Overwriting reducer1.py


In [19]:
!seq 1 10 | python mapper1.py

Total 1.0
Count 1
Total 2.0
Count 1
Total 3.0
Count 1
Total 4.0
Count 1
Total 5.0
Count 1
Total 6.0
Count 1
Total 7.0
Count 1
Total 8.0
Count 1
Total 9.0
Count 1
Total 10.0
Count 1


In [21]:
!seq 1  10 | python mapper1.py | python reducer1.py

Average 5.5


## ventas por producto

In [34]:
%%writefile data.txt
Fecha,Producto,Cantidad,PrecioUnitario
2023-10-23,ProductoA,5,10.0
2023-10-23,ProductoB,3,15.0
2023-10-24,ProductoA,2,10.0
2023-10-24,ProductoC,1,20.0
2023-10-25,ProductoB,4,15.0
2023-10-25,ProductoC,2,20.0
2023-10-26,ProductoA,3,10.0
2023-10-26,ProductoB,2,15.0
2023-10-27,ProductoC,3,20.0
2023-10-27,ProductoD,1,25.0


Overwriting data.txt


In [22]:
%%writefile mapper2.py

#!/usr/bin/env python
import sys

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    # Divide la línea en campos
    fields = line.strip().split(",")
    if len(fields) == 4:
        date, product, quantity, price = fields
        # Emite el producto como clave y la cantidad como valor
        print(f"{product}\t{quantity}")


Writing mapper2.py


In [29]:
%%writefile reducer2.py

#!/usr/bin/env python
import sys

current_product = None
total_quantity = 0

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    product, quantity = line.strip().split("\t")
    quantity = int(quantity)

    if product == current_product:
        total_quantity += quantity
    else:
        if current_product:
            print(f"{current_product}\t{total_quantity}")
        current_product = product
        total_quantity = quantity

if current_product:
    print(f"{current_product}\t{total_quantity}")


Overwriting reducer2.py


In [37]:
!cat data.txt | grep -v "Fecha" | python mapper2.py |python reducer2.py

ProductoA	5
ProductoB	3
ProductoA	2
ProductoC	1
ProductoB	4
ProductoC	2
ProductoA	3
ProductoB	2
ProductoC	3
ProductoD	1


## Puntajes de Peliculas.

In [40]:
%%writefile movie_data.txt
UserID,MovieID,Rating,Timestamp
1,101,4,964982703
1,102,3,964982703
1,103,5,964982703
2,101,5,964982703
2,103,2,964982703
2,104,4,964982703
3,102,4,964982703
3,104,3,964982703
3,105,2,964982703
4,101,3,964982703
4,105,5,964982703
4,106,4,964982703
5,102,4,964982703
5,106,5,964982703
5,107,2,964982703
6,101,4,964982703
6,103,3,964982703
6,108,5,964982703
7,102,5,964982703
7,108,4,964982703
7,109,3,964982703
8,101,2,964982703
8,103,4,964982703
8,110,5,964982703
9,102,4,964982703
9,110,3,964982703
9,111,2,964982703
10,101,5,964982703
10,111,3,964982703
10,112,4,964982703
11,103,5,964982703
11,112,4,964982703
11,113,3,964982703
12,101,2,964982703
12,113,5,964982703
12,114,4,964982703
13,102,3,964982703
13,114,5,964982703
13,115,4,964982703
14,101,4,964982703
14,103,5,964982703
14,115,3,964982703
15,102,2,964982703
15,116,4,964982703
15,117,5,964982703
16,101,3,964982703
16,116,4,964982703
16,118,5,964982703
17,102,4,964982703
17,118,3,964982703
17,119,2,964982703
18,101,5,964982703
18,103,4,964982703
18,119,3,964982703
19,102,3,964982703
19,119,4,964982703
19,120,5,964982703
20,101,2,964982703
20,103,5,964982703
20,120,4,964982703
21,102,4,964982703
21,120,5,964982703
21,121,3,964982703
22,101,3,964982703
22,121,5,964982703
22,122,4,964982703
23,102,5,964982703
23,122,3,964982703
23,123,2,964982703
24,101,4,964982703
24,123,5,964982703
24,124,3,964982703
25,102,2,964982703
25,124,4,964982703
25,125,5,964982703

Writing movie_data.txt


In [38]:
%%writefile mapper3.py

#!/usr/bin/env python
import sys

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    # Divide la línea en campos
    fields = line.strip().split(",")
    if len(fields) == 4:
        user, movie, rating, timestamp = fields
        # Emite el MovieID como clave y la calificación como valor
        print(f"{movie}\t{rating}")


Writing mapper3.py


In [39]:
%%writefile reducer3.py
#!/usr/bin/env python
import sys

current_movie = None
total_rating = 0
total_reviews = 0

# Lee líneas de entrada desde la entrada estándar
for line in sys.stdin:
    movie, rating = line.strip().split("\t")
    rating = int(rating)

    if movie == current_movie:
        total_rating += rating
        total_reviews += 1
    else:
        if current_movie:
            average_rating = total_rating / total_reviews
            print(f"{current_movie}\t{average_rating:.2f}")
        current_movie = movie
        total_rating = rating
        total_reviews = 1

if current_movie:
    average_rating = total_rating / total_reviews
    print(f"{current_movie}\t{average_rating:.2f}")


Writing reducer3.py


In [43]:
!cat movie_data.txt  | python mapper3.py |python reducer2.py

101	46
102	43
103	33
104	7
105	7
106	9
107	2
108	9
109	3
110	8
111	5
112	8
113	8
114	9
115	7
116	8
117	5
118	8
119	9
120	14
121	8
122	7
123	7
124	7
125	5


## Multiplicacion de matrices con Hadoop

In [58]:
!seq 1 100 | awk '{i++;print "A,"i","int(rand()*20); if(i==10){i=0}}' > matrix_in.txt
!seq 1 100 | awk '{i++;print "B,"i","int(rand()*20); if(i==10){i=0}}' >> matrix_in.txt
! head matrix_in.txt
! tail matrix_in.txt

A,1,6
A,2,12
A,3,17
A,4,1
A,5,8
A,6,10
A,7,10
A,8,0
A,9,12
A,10,5
B,1,2
B,2,8
B,3,4
B,4,2
B,5,15
B,6,9
B,7,5
B,8,7
B,9,12
B,10,3
